In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm
import itertools

from itertools import islice
from collections import Counter, defaultdict
from nltk.util import ngrams

## Novelty
- Si apparition de la classe principale alors qu'elle n'est pas dans les classes principales KB
- Si apparition d'une classe même parmis les secondaires

In [2]:
# utilisait clean et results... je ne les ai pas
toEval = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/toEval/2016_G06F_patents_toEval.csv')
KS = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/KS/2016_1115_G06F_KS_raw.csv')
print("toEval lenght: ", len(toEval))
print("KS lenght: ", len(KS))

toEval lenght:  19
KS lenght:  507


In [3]:
from itertools import chain
import ast

toEval_ipc = list([ast.literal_eval(i) for i in toEval.sec_ipc])
KS_ipc = list(set(chain.from_iterable([ast.literal_eval(s) for s in KS.sec_ipc])))

print(toEval_ipc)
print(KS_ipc)
print("toEval length ipc: ", len(toEval_ipc))
print("KS length of unique ipc: ", len(KS_ipc))

[['G06F1900', 'G06F1718'], ['G06F306', 'G06F1342', 'G06F1208', 'G06F1328', 'G06F944'], ['G06F1114', 'G06F2162', 'G06F1730'], ['G06F314'], ['G06F944'], ['G06F952', 'G06F950'], ['G06F1730'], ['G06F118'], ['G06F312', 'H04N132', 'H04N121', 'H04N100'], ['G06F303', 'G06F3041', 'G06F116'], ['G06F306', 'G06F1210'], ['G06F1730'], ['G06F1214', 'G11C710'], ['G06F306', 'G11C114091', 'G11C114096', 'G11C1400', 'G11C1116'], ['G06F2136', 'G06F30488', 'G06F301', 'G06F2131'], ['G06F306', 'G06F312'], ['G06F1730'], ['G06F1128', 'G06F11263'], ['G06F1316', 'G11C114093', 'G06F1340']]
['G06F15177', 'G06Q2024', 'G06Q5024', 'G06F1216', 'G06F950', 'G06F302', 'G06F3023', 'G06F2150', 'G06F945', 'G06F1314', 'G06F930', 'G06F1110', 'G06T1380', 'G06F946', 'H04R500', 'G06F952', 'A61B5117', 'G06F1750', 'G06F954', 'G06F1202', 'G06F1516', 'G06F3045', 'H04N715', 'G06F3038', 'H05K102', 'G08B600', 'G06F9312', 'H05K700', 'G06F1114', 'G06F1130', 'G06F3044', 'G01P1500', 'H04W2400', 'G06F1722', 'G11C2952', 'G06F1724', 'A61B500',

In [4]:
toEval.sec_ipc[0]
[ast.literal_eval(i)[0] for i in toEval.sec_ipc]
print(KS.sec_ipc[2])
[ast.literal_eval(s)[0] for s in KS.sec_ipc]

['G06F116', 'G06F126', 'G06F1516']


['G06F1202',
 'G06F1516',
 'G06F116',
 'G06F3041',
 'G06F1750',
 'G06F3048',
 'G06F316',
 'G06F3048',
 'G06F301',
 'G06F1516',
 'G06F300',
 'G06F1516',
 'G06F1500',
 'G06F1730',
 'G06F1730',
 'G06F3041',
 'G06F2100',
 'G06F1500',
 'G06F938',
 'G06F1730',
 'G06F1516',
 'G06F1730',
 'G06F1208',
 'G06F1516',
 'G06F1718',
 'G06F1750',
 'G06F700',
 'G06F704',
 'G06F1900',
 'G06F945',
 'G06F312',
 'G06F3041',
 'G06F1724',
 'G06F1516',
 'G06F1208',
 'G06F930',
 'G06F1730',
 'G06F944',
 'G06F3041',
 'G06F312',
 'G06F15173',
 'G06F1730',
 'G06F1116',
 'G06F1730',
 'G06F944',
 'G06F1900',
 'G06F1100',
 'G06F1516',
 'G06F132',
 'G06F132',
 'G06F1516',
 'G06F312',
 'G06F9455',
 'G06F300',
 'G06F114',
 'G06F1516',
 'G06F3045',
 'G06F1727',
 'G06F1516',
 'G06F126',
 'G06F1216',
 'G06F9445',
 'G06F15173',
 'G06F944',
 'G06F1206',
 'G06F1711',
 'G06F301',
 'G06F2100',
 'G06F9312',
 'G06F2100',
 'G06F1730',
 'G06F1727',
 'G06F3033',
 'G06F132',
 'G06F1900',
 'G06F1720',
 'G06F2100',
 'G06F1516',
 'G06F

In [5]:
toEval_mainIPC = list([ast.literal_eval(i)[0] for i in toEval.sec_ipc])
KS_mainIPC = list(set([ast.literal_eval(s)[0] for s in KS.sec_ipc]))
test_KS_mainIPC = list(set(KS.ipc))

print(toEval_mainIPC)
print(KS_mainIPC)
print("toEval length main ipc: ", len(toEval_mainIPC))
print("KS length of unique main ipc: ", len(KS_mainIPC))

# Vérif
# KS_mainIPC == test_KS_mainIPC

['G06F1900', 'G06F306', 'G06F1114', 'G06F314', 'G06F944', 'G06F952', 'G06F1730', 'G06F118', 'G06F312', 'G06F303', 'G06F306', 'G06F1730', 'G06F1214', 'G06F306', 'G06F2136', 'G06F306', 'G06F1730', 'G06F1128', 'G06F1316']
['G06F1216', 'G06F950', 'G06F302', 'G06F945', 'G06F1314', 'G06F930', 'G06F1110', 'G06F946', 'G06F952', 'G06F1750', 'G06F954', 'G06F1202', 'G06F1516', 'G06F3045', 'G06F3038', 'G06F9312', 'G06F1114', 'G06F1130', 'G06F3044', 'G06F1722', 'G06F1724', 'G06F1720', 'G06F944', 'G06F1714', 'G06F2178', 'G06F1730', 'G06F1715', 'G06F700', 'G06F1208', 'G06F1518', 'G06F1711', 'G06F2134', 'G06F1116', 'G06F9445', 'G06F1126', 'G06F2170', 'G06F1312', 'G06F1134', 'G06F301', 'G06F704', 'G06F1206', 'G06F2100', 'G06F1210', 'G06F1100', 'G06F118', 'G06F120', 'G06F724', 'G06F112', 'G06F1342', 'G06F2160', 'G06F116', 'G06F3042', 'G06F132', 'G06F1928', 'G06F9455', 'G06F1324', 'G06F1212', 'G06F30486', 'G06F1328', 'G06F938', 'G06F30346', 'G06F1700', 'G06F2162', 'G06F2157', 'G06F30484', 'G06F1340', 'G0

In [6]:
# print(list(KS.ipc))
# print(list(KS.ipc))

score = []
for patent in range(0, len(toEval_ipc)):
    novelty = 0
    for i in range(0, len(toEval_ipc[patent])):
        if toEval_ipc[patent][i] not in KS_ipc:
            novelty += 1
    score.append(novelty/len(toEval_ipc[patent]))

print(score) 

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3333333333333333, 0.0, 0.0, 0.5, 0.8, 0.25, 0.0, 0.0, 1.0, 0.3333333333333333]


In [7]:
# toEval_mainIPC[3]
# KS_mainIPC

In [8]:
score_main = []
for patent in range(0, len(toEval_mainIPC)):
    novelty = 0
    if toEval_mainIPC[patent] not in KS_mainIPC:
        # print(toEval_mainIPC[patent])
        novelty += 1
    score_main.append(novelty)

print(score_main) 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1]


In [9]:
df = pd.DataFrame({
    'All IPCs': [round(i,2) for i in score],  # Renaming the first column
    'Main IPCs': score_main   # Renaming the second column
})
df

,All IPCs,Main IPCs
0,0.00,0
1,0.00,0
2,0.00,0
3,0.00,0
4,0.00,0
5,0.00,0
6,0.00,0
7,0.00,0
8,0.50,0
9,0.33,1


## Unique
- Création brevet prototypique pour KS
    - Par IPC et année
Prendre les n premiers IPC

In [10]:
pd.DataFrame([ast.literal_eval(i) for i in KS.sec_ipc])

,0,1,2,3,4,5,6
0,G06F1202,None,None,None,None,None,None
1,G06F1516,None,None,None,None,None,None
2,G06F116,G06F126,G06F1516,None,None,None,None
3,G06F3041,None,None,None,None,None,None
4,G06F1750,None,None,None,None,None,None
...,...,...,...,...,...,...,...
502,G06F944,G06F930,None,None,None,None,None
503,G06F1107,None,None,None,None,None,None
504,G06F1715,None,None,None,None,None,None
505,G06F3044,G06F3041,None,None,None,None,None


In [11]:
type(KS.sec_ipc[0])
KS.sec_ipc = KS.sec_ipc.apply(ast.literal_eval)

df = pd.DataFrame(KS.sec_ipc, columns=['sec_ipc'])
df

# # Step 2: Flatten the list of lists into a single list
flat_list = [item for sublist in df['sec_ipc'] for item in sublist]
flat_list
# # Step 3: Count occurrences using Counter
occurrences = Counter(flat_list)

# # Step 4: Convert to DataFrame for easier viewing (optional)
occurrences_df = pd.DataFrame(occurrences.items(), columns=['IPC', 'Count'])


# pd.reset_option('display.max_rows')
# pd.set_option('display.width', 1000) 
occurrences_df.sort_values(by='Count', ascending=False).reset_index(drop=True)[0:30]


,IPC,Count
0,G06F1730,75
1,G06F1516,40
2,G06F3041,33
3,G06F1750,27
4,G06F1900,21
5,G06F301,20
6,G06F30484,17
7,G06F944,14
8,G06F3044,13
9,G06F2100,12


In [24]:
occurrences_df.sort_values(by='Count', ascending=False).reset_index(drop=True)[0:5].IPC
toEval

0    G06F1730
1    G06F1516
2    G06F3041
3    G06F1750
4    G06F1900
Name: IPC, dtype: object

In [61]:
from sklearn.metrics import jaccard_score

protoPatent = set(list(occurrences_df.sort_values(by='Count', ascending=False)[0:5].IPC))
print("toEval patent IPCs: ", toEval_ipc[2])
print("Prototypical patent IPCs: ", protoPatent)

jaccard_index =[]
for i in range(0, len(toEval_ipc)):
    intersection = protoPatent.intersection(toEval_ipc[i])
    union = protoPatent.union(toEval_ipc[i])
    jaccard_index.append(len(intersection) / len(union))

jaccard_index

toEval patent IPCs:  ['G06F1114', 'G06F2162', 'G06F1730']
Prototypical patent IPCs:  {'G06F3041', 'G06F1750', 'G06F1516', 'G06F1900', 'G06F1730'}


[0.16666666666666666,
 0.0,
 0.14285714285714285,
 0.0,
 0.0,
 0.0,
 0.2,
 0.0,
 0.0,
 0.14285714285714285,
 0.0,
 0.2,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2,
 0.0,
 0.0]